#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 4, 28)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,20,B,KTB,"5,000",14.80,1,1L,SET50,2022-02-02


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
3,B,MCS,-3,ROUND,SET,10000,12.000000,12.300000,0.000000,0.00%,2,2022-02-02,Buy,120000.000000
5,B,TISCO,-3,ROUND,SET50,1000,99.000000,99.750000,0.750000,+0.76%,2,2022-02-02,Buy,99000.000000
7,S,DIF,1,DOS,SET,10000,14.200000,14.100000,-0.100000,-0.70%,2,2022-02-02,Sell,142000.000000
8,S,DOHOME,6,RD15pct,SET100,2400,20.900000,20.300000,-0.600000,-2.87%,2,2022-02-02,Sell,50160.000000
11,S,JASIF,1,DOS,SET,10000,11.000000,10.900000,0.000000,0.00%,2,2022-03-03,Sell,110000.000000


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index()

trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
B,MAKRO,-2,RD05pct,SET,1500,36.000000,36.500000,-1.000000,-2.67%,1,2022-03-03,Buy,54000.000000
B,BAM,-3,3L,SET100,12000,19.800000,20.100000,0.200000,+1.01%,1,2022-04-29,Buy,237600.000000
B,MCS,-3,ROUND,SET,10000,12.000000,12.300000,0.000000,0.00%,2,2022-02-02,Buy,120000.000000
B,SC,-5,HD,SET,30000,3.800000,3.900000,0.020000,+0.52%,1,2022-04-29,Buy,114000.000000
B,KKP,-8,3L,SET100,3000,70.000000,72.000000,-2.500000,-3.36%,1,2022-04-28,Buy,210000.000000


### End of Tables in the process

### Print to verify before upload file

In [7]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [8]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [9]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,DOHOME,"2,400",20.00,"48,000.00",ROUND,SET100,2,2022-02-02
1,B,MCS,"10,000",12.00,"120,000.00",ROUND,SET,2,2022-02-02
2,S,DIF,"10,000",14.20,"142,000.00",DOS,SET,2,2022-02-02
3,S,JASIF,"10,000",11.00,"110,000.00",DOS,SET,2,2022-03-03


In [10]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    168000.0
S    252000.0
Name: amount, dtype: float64

In [11]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,DOHOME,"2,400",20.00,"48,000.00",ROUND,SET100,2,2022-02-02
1,B,MCS,"10,000",12.00,"120,000.00",ROUND,SET,2,2022-02-02


In [12]:
cash = 210_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(210000, 168000.0, 42000.0)

In [14]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
2,S,DIF,"10,000",14.20,"142,000.00",DOS,SET,2,2022-02-02
3,S,DOHOME,"2,400",20.90,"50,160.00",RD15pct,SET100,2,2022-02-02
4,S,JASIF,"10,000",11.00,"110,000.00",DOS,SET,2,2022-03-03


In [15]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(210000, 302160.0, 512160.0)

### After call ord-log (must call everytime that orders change)

In [16]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [17]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [18]:
mb = (df.trans == 'Buy') & (df.spd >= -3)
df[mb].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,BAM,-3,3L,SET100,12000,19.8,20.10,0.20,+1.01%,1,2022-04-29,Buy,237600.0
2,B,MAKRO,-2,RD05pct,SET,1500,36.0,36.50,-1.00,-2.67%,1,2022-03-03,Buy,54000.0
3,B,MCS,-3,ROUND,SET,10000,12.0,12.30,0.00,0.00%,2,2022-02-02,Buy,120000.0
5,B,TISCO,-3,ROUND,SET50,1000,99.0,99.75,0.75,+0.76%,2,2022-02-02,Buy,99000.0


In [19]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [20]:
ms = (df.trans == 'Sell') & (df.spd <= 4)
df[ms].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
7,S,DIF,1,DOS,SET,10000,14.2,14.1,-0.1,-0.70%,2,2022-02-02,Sell,142000.0
11,S,JASIF,1,DOS,SET,10000,11.0,10.9,0.0,0.00%,2,2022-03-03,Sell,110000.0


In [21]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,BAM,-3,3L,SET100,12000,19.8,20.10,0.20,+1.01%,1,2022-04-29,Buy,237600.0
2,B,MAKRO,-2,RD05pct,SET,1500,36.0,36.50,-1.00,-2.67%,1,2022-03-03,Buy,54000.0
3,B,MCS,-3,ROUND,SET,10000,12.0,12.30,0.00,0.00%,2,2022-02-02,Buy,120000.0
5,B,TISCO,-3,ROUND,SET50,1000,99.0,99.75,0.75,+0.76%,2,2022-02-02,Buy,99000.0
7,S,DIF,1,DOS,SET,10000,14.2,14.10,-0.10,-0.70%,2,2022-02-02,Sell,142000.0
11,S,JASIF,1,DOS,SET,10000,11.0,10.90,0.00,0.00%,2,2022-03-03,Sell,110000.0


In [22]:
df[mb | ms].shape[0]

6

In [23]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
0,BAM,Buy,"237,600.00",-3
1,KKP,Buy,"210,000.00",-8
3,MCS,Buy,"120,000.00",-3
4,SC,Buy,"114,000.00",-5
5,TISCO,Buy,"99,000.00",-3


### Select source of orders between these two

In [24]:
df_out = df[mb | ms][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
0,Buy,BAM,12000,19.8,1,-3,20.10,0.20,+1.01%,3L,SET100,2022-04-29
2,Buy,MAKRO,1500,36.0,1,-2,36.50,-1.00,-2.67%,RD05pct,SET,2022-03-03
3,Buy,MCS,10000,12.0,2,-3,12.30,0.00,0.00%,ROUND,SET,2022-02-02
5,Buy,TISCO,1000,99.0,2,-3,99.75,0.75,+0.76%,ROUND,SET50,2022-02-02
7,Sell,DIF,10000,14.2,2,1,14.10,-0.10,-0.70%,DOS,SET,2022-02-02
11,Sell,JASIF,10000,11.0,2,1,10.90,0.00,0.00%,DOS,SET,2022-03-03


In [25]:
df_out = df[df.active == 2][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
3,Buy,MCS,10000,12.0,2,-3,12.30,0.00,0.00%,ROUND,SET,2022-02-02
5,Buy,TISCO,1000,99.0,2,-3,99.75,0.75,+0.76%,ROUND,SET50,2022-02-02
7,Sell,DIF,10000,14.2,2,1,14.10,-0.10,-0.70%,DOS,SET,2022-02-02
8,Sell,DOHOME,2400,20.9,2,6,20.30,-0.60,-2.87%,RD15pct,SET100,2022-02-02
11,Sell,JASIF,10000,11.0,2,1,10.90,0.00,0.00%,DOS,SET,2022-03-03


In [26]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process